In [1]:
import sys
sys.version_info

sys.version_info(major=3, minor=11, micro=5, releaselevel='final', serial=0)

In [2]:
!pip freeze

absl-py==1.4.0
aiobotocore @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_71xswk40o_/croot/aiobotocore_1682537536268/work
aiodns==2.0.0
aiofiles @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_f56ag8l7kr/croot/aiofiles_1683773599608/work
aiohttp @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_6fjtz5frfh/croot/aiohttp_1701112546765/work
aioitertools @ file:///tmp/build/80754af9/aioitertools_1607109665762/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
aiosqlite @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_3d75lecab1/croot/aiosqlite_1683773918307/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-anon-usage @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_ca68ugx5h9/croot/anaconda-anon-usage_1697038938120/work
anaconda-catalogs @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_e8tmw882qa/croot/anaconda

In [ ]:
import pandas as pd

df = pd.read_excel("Table S1.xlsx")

In [ ]:
df.replace('-', None, inplace=True)
df.replace('na', None, inplace=True)
df.replace('unknown', None, inplace=True)

In [ ]:
df

In [ ]:
min(df["Age"])

In [ ]:
df["Sex"].value_counts()

In [ ]:
df["ET status"].value_counts()

In [ ]:
df["PD status"].value_counts()

In [ ]:
df["Clinical Assesment"].value_counts()

In [ ]:
df["ET Severity"].value_counts()

In [ ]:
df["Onset"].value_counts()

In [ ]:
df["Status"].value_counts()

In [ ]:
df['survival_event'] = df['Status'].apply(lambda x: 1 if x == 'deceased' else 0)

In [ ]:
df

In [ ]:
survival_df = df[["Age","Sex","ET status","survival_event"]]
survival_df

In [ ]:
survival_df['ET status'] = survival_df['ET status'].map({'yes': 1, 'no': 0})
survival_df['Sex'] = survival_df['Sex'].map({'female': 1, 'male': 2})

In [ ]:
survival_df

In [ ]:
from lifelines.statistics import logrank_test
results = logrank_test(survival_df[survival_df["ET status"] == 0]["Age"],
                       survival_df[survival_df["ET status"] == 1]["Age"], 
                       event_observed_A=survival_df[survival_df["ET status"] == 0]["survival_event"], 
                       event_observed_B=survival_df[survival_df["ET status"] == 1]["survival_event"])

results.print_summary()
print(results.p_value)       
print(results.test_statistic) 

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(survival_df[["Age","survival_event","ET status"]], duration_col='Age', event_col='survival_event')
cph.print_summary()

In [ ]:
hr = cph.hazard_ratios_
hr_sum = cph.summary
print(hr[0],hr_sum)

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.statistics import logrank_test
import unicodeit
import numpy as np
import matplotlib.pyplot as plt

ax = plt.subplot(111)

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("All Cohort")
#plt.xlabel("Age (Years)")
plt.ylabel("Survival Probability")
plt.legend(loc='best')

kmf_no = KaplanMeierFitter()
ax = kmf_no.fit(survival_df[survival_df["ET status"] == 0]["Age"],
                survival_df[survival_df["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes = KaplanMeierFitter()
ax = kmf_yes.fit(survival_df[survival_df["ET status"] == 1]["Age"],
                 survival_df[survival_df["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no, kmf_yes, ax=ax)

# Calculate and annotate median survival times
median_survival_et = kmf_yes.median_survival_time_
median_survival_non_et = kmf_no.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=77,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.715 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(survival_df[survival_df["ET status"] == 0]["Age"],
                       survival_df[survival_df["ET status"] == 1]["Age"], 
                       event_observed_A=survival_df[survival_df["ET status"] == 0]["survival_event"], 
                       event_observed_B=survival_df[survival_df["ET status"] == 1]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}^{digits[1]}")

plt.text(40, 0.3, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(survival_df[["Age","survival_event","ET status"]], duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(40, 0.2, 
         f"HR : {round(hr[0],ndigits=2)}, $CI_{{95\%}}:$[{round(hr_sum.loc['ET status', 'exp(coef) lower 95%'],ndigits=2)},{round(hr_sum.loc['ET status', 'exp(coef) upper 95%'],ndigits=2)}]", 
         fontsize=10)

ax.set_xlabel("Age (Years)")

plt.show()

In [ ]:
kmf_no.median_survival_time_

In [ ]:
kmf_yes.median_survival_time_

In [ ]:
et_positive_df = survival_df[survival_df["ET status"]==1]
et_negative_df = survival_df[survival_df["ET status"]==0]
female_df = survival_df[survival_df["Sex"]==1]
male_df = survival_df[survival_df["Sex"]==2]

In [ ]:
from lifelines.statistics import logrank_test
results = logrank_test(et_positive_df[et_positive_df["Sex"] == 1]["Age"],
                       et_positive_df[et_positive_df["Sex"] == 2]["Age"], 
                       event_observed_A=et_positive_df[et_positive_df["Sex"] == 1]["survival_event"], 
                       event_observed_B=et_positive_df[et_positive_df["Sex"] == 2]["survival_event"])

results.print_summary()
print(results.p_value)       
print(results.test_statistic) 

In [ ]:
round(results.p_value,ndigits=3)

In [ ]:
et_positive_df

In [ ]:
et_positive_cox = et_positive_df
et_positive_cox = et_positive_cox.drop("ET status",axis=1)
et_positive_cox

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(et_positive_cox, duration_col='Age', event_col='survival_event')
cph.print_summary()

In [ ]:
hr = cph.hazard_ratios_
hr_sum = cph.summary
print(hr[0],hr_sum)

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.statistics import logrank_test
import unicodeit
import numpy as np
import matplotlib.pyplot as plt

ax = plt.subplot(111)

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("ET+")
plt.xlabel("Age (Years)")
plt.ylabel("Survival Probability")
plt.legend(loc='best')

kmf_female = KaplanMeierFitter()
ax = kmf_female.fit(et_positive_df[et_positive_df["Sex"] == 1]["Age"],
                et_positive_df[et_positive_df["Sex"] == 1]["survival_event"], 
                label='Female').plot_survival_function(ax=ax,color="#DC0000B2")

kmf_male = KaplanMeierFitter()
ax = kmf_male.fit(et_positive_df[et_positive_df["Sex"] == 2]["Age"],
                 et_positive_df[et_positive_df["Sex"] == 2]["survival_event"], 
                 label='Male').plot_survival_function(ax=ax,color="#4DBBD5B2")

add_at_risk_counts(kmf_female, kmf_male, ax=ax)

# Calculate and annotate median survival times
median_survival_female = kmf_female.median_survival_time_
median_survival_male = kmf_male.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.71 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(et_positive_df[et_positive_df["Sex"] == 1]["Age"],
                       et_positive_df[et_positive_df["Sex"] == 2]["Age"], 
                       event_observed_A=et_positive_df[et_positive_df["Sex"] == 1]["survival_event"], 
                       event_observed_B=et_positive_df[et_positive_df["Sex"] == 2]["survival_event"])


logrank_p_value = round(results.p_value,ndigits=3)

plt.text(40, 0.3, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(et_positive_cox, duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(40, 0.2, 
         f"HR : {round(hr[0],ndigits=2)}, $CI_{{95\%}}:$[{round(hr_sum.loc['Sex', 'exp(coef) lower 95%'],ndigits=2)},{round(hr_sum.loc['Sex', 'exp(coef) upper 95%'],ndigits=2)}]", 
         fontsize=10)

plt.show()

In [ ]:
median_survival_female

In [ ]:
median_survival_male

In [ ]:
from lifelines.statistics import logrank_test
results = logrank_test(et_negative_df[et_negative_df["Sex"] == 1]["Age"],
                       et_negative_df[et_negative_df["Sex"] == 2]["Age"], 
                       event_observed_A=et_negative_df[et_negative_df["Sex"] == 1]["survival_event"], 
                       event_observed_B=et_negative_df[et_negative_df["Sex"] == 2]["survival_event"])

results.print_summary()
print(results.p_value)       
print(results.test_statistic) 

In [ ]:
number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}")
logrank_p_value

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(et_negative_df[["Age","Sex","survival_event"]], duration_col='Age', event_col='survival_event')
cph.print_summary()

In [ ]:
hr = cph.hazard_ratios_
hr_sum = cph.summary
print(hr[0],hr_sum)

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import unicodeit
import numpy as np
import matplotlib.pyplot as plt

ax = plt.subplot(111)

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("ET-")
plt.xlabel("Age (Years)")
plt.ylabel("Survival Probability")
plt.legend(loc='best')

kmf_female = KaplanMeierFitter()
ax = kmf_female.fit(et_negative_df[et_negative_df["Sex"] == 1]["Age"],
                et_negative_df[et_negative_df["Sex"] == 1]["survival_event"], 
                label='Female').plot_survival_function(ax=ax,color="#DC0000B2")

kmf_male = KaplanMeierFitter()
ax = kmf_male.fit(et_negative_df[et_negative_df["Sex"] == 2]["Age"],
                 et_negative_df[et_negative_df["Sex"] == 2]["survival_event"], 
                 label='Male').plot_survival_function(ax=ax,color="#4DBBD5B2")

add_at_risk_counts(kmf_female, kmf_male, ax=ax)

# Calculate and annotate median survival times
median_survival_female = kmf_female.median_survival_time_
median_survival_male = kmf_male.median_survival_time_

ax.axhline(y=0.5,xmax=0.45 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=79,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.62 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=73,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(et_negative_df[et_negative_df["Sex"] == 1]["Age"],
                       et_negative_df[et_negative_df["Sex"] == 2]["Age"], 
                       event_observed_A=et_negative_df[et_negative_df["Sex"] == 1]["survival_event"], 
                       event_observed_B=et_negative_df[et_negative_df["Sex"] == 2]["survival_event"])


logrank_p_value = round(results.p_value,ndigits=4)

plt.text(40, 0.3, f"Log-rank p-value : {logrank_p_value}", fontsize=10)



cph = CoxPHFitter()
cph.fit(et_negative_df[["Age","Sex","survival_event"]], duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(40, 0.2, 
         f"HR : {round(hr[0],ndigits=2)}, $CI_{{95\%}}:$[{round(hr_sum.loc['Sex', 'exp(coef) lower 95%'],ndigits=2)},{round(hr_sum.loc['Sex', 'exp(coef) upper 95%'],ndigits=2)}]", 
         fontsize=10)

plt.show()

In [ ]:
median_survival_female

In [ ]:
median_survival_male

In [ ]:
from lifelines.statistics import logrank_test
results = logrank_test(male_df[male_df["ET status"] == 1]["Age"],
                       male_df[male_df["ET status"] == 0]["Age"], 
                       event_observed_A=male_df[male_df["ET status"] == 1]["survival_event"], 
                       event_observed_B=male_df[male_df["ET status"] == 0]["survival_event"])

results.print_summary()
print(results.p_value)       
print(results.test_statistic) 

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Replace 'ET status' with the actual column name if it's different in your DataFrame

# Convert 'ET status' to categorical for better representation
survival_df['ET status'] = survival_df['ET status'].astype('category')

# Create dummy variables for 'ET status'
survival_df_dummies = pd.get_dummies(survival_df['ET status'], prefix='status',dtype="int")

# Concatenate the dummy variables with the original DataFrame
survival_df_expanded = pd.concat([survival_df, survival_df_dummies], axis=1)

# Drop the original 'ET status' column if you no longer need it
survival_df_expanded = survival_df_expanded.drop('ET status', axis=1)

# Print the expanded DataFrame
survival_df_expanded

In [ ]:
column_mapping = {"status_0":"No","status_1":"Yes"}

# Rename the columns using the dictionary
survival_df_expanded = survival_df_expanded.rename(columns=column_mapping)

In [ ]:
survival_df_expanded # ET status yes ve no diye ayır o zaman multivariate olacak

In [ ]:
male_cox = survival_df_expanded[survival_df_expanded["Sex"]==2]
male_cox = male_cox.drop("Sex",axis=1)
male_cox

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter(penalizer=0.1)
cph.fit(male_cox, duration_col='Age', event_col='survival_event',formula="Yes")
cph.print_summary()

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter(penalizer=0.1)
cph.fit(male_cox, duration_col='Age', event_col='survival_event',formula="Yes+No")
cph.print_summary()

In [ ]:
hr = cph.hazard_ratios_
hr_sum = cph.summary
print(hr[0],hr_sum)

In [ ]:
hr["Yes"]

In [ ]:
hr["No"]

In [ ]:
hr_sum

In [ ]:
hr_sum.loc['Yes', "exp(coef) lower 95%"]

In [ ]:
hr_sum.loc['Yes', "exp(coef) upper 95%"]

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.statistics import logrank_test
import unicodeit
import numpy as np
import matplotlib.pyplot as plt

ax = plt.subplot(111)

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("Male")
plt.xlabel("Age (Years)")
plt.ylabel("Survival Probability")
plt.legend(loc='best')

kmf_no_male = KaplanMeierFitter()
ax = kmf_no_male.fit(male_df[male_df["ET status"] == 0]["Age"],
                male_df[male_df["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes_male = KaplanMeierFitter()
ax = kmf_yes_male.fit(male_df[male_df["ET status"] == 1]["Age"],
                 male_df[male_df["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no_male, kmf_yes_male, ax=ax)

# Calculate and annotate median survival times
median_survival_et_male = kmf_yes_male.median_survival_time_
median_survival_non_et_male = kmf_no_male.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=73,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.71 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(male_df[male_df["ET status"] == 1]["Age"],
                       male_df[male_df["ET status"] == 0]["Age"], 
                       event_observed_A=male_df[male_df["ET status"] == 1]["survival_event"], 
                       event_observed_B=male_df[male_df["ET status"] == 0]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}^{digits[1]}")

plt.text(39, 0.38, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

cph = CoxPHFitter(penalizer=0.1)
cph.fit(male_cox, duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(39, 0.23, 
         f"ET(+) HR : {round(hr['Yes'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['Yes', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['Yes', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)

plt.text(39, 0.08, 
         f"ET(-) HR : {round(hr['No'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['No', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['No', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)

plt.show()

In [ ]:
median_survival_et_male

In [ ]:
median_survival_non_et_male

In [ ]:
from lifelines.statistics import logrank_test
results = logrank_test(female_df[female_df["ET status"] == 1]["Age"],
                       female_df[female_df["ET status"] == 0]["Age"], 
                       event_observed_A=female_df[female_df["ET status"] == 1]["survival_event"], 
                       event_observed_B=female_df[female_df["ET status"] == 0]["survival_event"])

results.print_summary()
print(results.p_value)       
print(results.test_statistic) 

In [ ]:
female_cox = survival_df_expanded[survival_df_expanded["Sex"]==1]
female_cox = female_cox.drop("Sex",axis=1)
female_cox

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter(penalizer=0.1)
cph.fit(female_cox, duration_col='Age', event_col='survival_event',formula="Yes+No")
cph.print_summary()

In [ ]:
hr = cph.hazard_ratios_
hr_sum = cph.summary
print(hr[0],hr_sum)

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.statistics import logrank_test
import unicodeit
import numpy as np
import matplotlib.pyplot as plt

ax = plt.subplot(111)

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("Female")
plt.xlabel("Age (Years)")
plt.ylabel("Survival Probability")
plt.legend(loc='best')

kmf_no_female = KaplanMeierFitter()
ax = kmf_no_female.fit(female_df[female_df["ET status"] == 0]["Age"],
                female_df[female_df["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes_female = KaplanMeierFitter()
ax = kmf_yes_female.fit(female_df[female_df["ET status"] == 1]["Age"],
                 female_df[female_df["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no_female, kmf_yes_female, ax=ax)

# Calculate and annotate median survival times
median_survival_et_female = kmf_yes_female.median_survival_time_
median_survival_non_et_female = kmf_no_female.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=79,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.71 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(female_df[female_df["ET status"] == 1]["Age"],
                       female_df[female_df["ET status"] == 0]["Age"], 
                       event_observed_A=female_df[female_df["ET status"] == 1]["survival_event"], 
                       event_observed_B=female_df[female_df["ET status"] == 0]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}")

plt.text(39, 0.38, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

cph = CoxPHFitter(penalizer=0.1)
cph.fit(female_cox, duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(39, 0.23, 
         f"ET(+) HR : {round(hr['Yes'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['Yes', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['Yes', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)

plt.text(39, 0.08, 
         f"ET(-) HR : {round(hr['No'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['No', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['No', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)

plt.show()

In [ ]:
median_survival_et_female

In [ ]:
median_survival_non_et_female

In [ ]:
survival_df_60 = survival_df[survival_df["Age"]>59]
survival_df_60

In [ ]:
min(survival_df_60["Age"])

In [ ]:
from lifelines.statistics import logrank_test
results = logrank_test(survival_df_60[survival_df_60["ET status"] == 0]["Age"],
                       survival_df_60[survival_df_60["ET status"] == 1]["Age"], 
                       event_observed_A=survival_df_60[survival_df_60["ET status"] == 0]["survival_event"], 
                       event_observed_B=survival_df_60[survival_df_60["ET status"] == 1]["survival_event"])

results.print_summary()
print(results.p_value)       
print(results.test_statistic) 

In [ ]:
from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(survival_df_60[["Age","survival_event","ET status"]], duration_col='Age', event_col='survival_event')
cph.print_summary()

In [ ]:
hr = cph.hazard_ratios_
hr_ci = cph.confidence_intervals_
print(hr[0],hr_ci)

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.statistics import logrank_test
import unicodeit
import numpy as np
import matplotlib.pyplot as plt

ax = plt.subplot(111)

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([55, 105])
plt.ylim([0, 1.05])

plt.title("All Cohort (Elderly)")
plt.xlabel("Age (Years)")
plt.ylabel("Survival Probability")
plt.legend(loc='best')

kmf_no = KaplanMeierFitter()
ax = kmf_no.fit(survival_df_60[survival_df_60["ET status"] == 0]["Age"],
                survival_df_60[survival_df_60["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes = KaplanMeierFitter()
ax = kmf_yes.fit(survival_df_60[survival_df_60["ET status"] == 1]["Age"],
                 survival_df_60[survival_df_60["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no, kmf_yes, ax=ax)

# Calculate and annotate median survival times
median_survival_et = kmf_yes.median_survival_time_
median_survival_non_et = kmf_no.median_survival_time_

ax.axhline(y=0.5,xmax=0.5 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=78,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.6 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(survival_df_60[survival_df_60["ET status"] == 0]["Age"],
                       survival_df_60[survival_df_60["ET status"] == 1]["Age"], 
                       event_observed_A=survival_df_60[survival_df_60["ET status"] == 0]["survival_event"], 
                       event_observed_B=survival_df_60[survival_df_60["ET status"] == 1]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}^{digits[1]}")

plt.text(59, 0.3, f"Log-rank\np-value : {logrank_p_value}", fontsize=10)

from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(survival_df_60[["Age","survival_event","ET status"]], duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_ci = cph.confidence_intervals_

plt.text(59, 0.18, 
         f"HR : {round(hr[0],ndigits=2)}\n$CI_{{95\%}}:$[{round(hr_ci.loc['ET status', '95% lower-bound'],ndigits=2)},{round(hr_ci.loc['ET status', '95% upper-bound'],ndigits=2)}]", 
         fontsize=10)

plt.show()

In [ ]:
median_survival_et

In [ ]:
median_survival_non_et

In [ ]:
from matplotlib.gridspec import GridSpec

fig = plt.figure(layout="constrained",figsize=(12,18))

gs = GridSpec(3, 2, figure=fig)

# All Cohort
ax = fig.add_subplot(gs[0, 0])

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("All Cohort")

plt.ylabel("Survival Probability")
plt.legend(loc='best')

ax.text(-0.03, 1.1, "A", transform=ax.transAxes, fontsize=12, fontweight='bold', va='top')

kmf_no = KaplanMeierFitter()
ax = kmf_no.fit(survival_df[survival_df["ET status"] == 0]["Age"],
                survival_df[survival_df["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes = KaplanMeierFitter()
ax = kmf_yes.fit(survival_df[survival_df["ET status"] == 1]["Age"],
                 survival_df[survival_df["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no, kmf_yes, ax=ax)

# Calculate and annotate median survival times
median_survival_et = kmf_yes.median_survival_time_
median_survival_non_et = kmf_no.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=77,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.715 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(survival_df[survival_df["ET status"] == 0]["Age"],
                       survival_df[survival_df["ET status"] == 1]["Age"], 
                       event_observed_A=survival_df[survival_df["ET status"] == 0]["survival_event"], 
                       event_observed_B=survival_df[survival_df["ET status"] == 1]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}^{digits[1]}")

plt.text(39, 0.3, f"Log-rank p-value : {logrank_p_value}", fontsize=10)


from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(survival_df[["Age","survival_event","ET status"]], duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(39, 0.2, 
         f"HR : {round(hr[0],ndigits=2)}, $CI_{{95\%}}:$[{round(hr_sum.loc['ET status', 'exp(coef) lower 95%'],ndigits=2)},{round(hr_sum.loc['ET status', 'exp(coef) upper 95%'],ndigits=2)}]", 
         fontsize=10)
ax.set_xlabel("Age (Years)")

# Elderly Cohort
ax = fig.add_subplot(gs[0, 1])

plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([55, 105])
plt.ylim([0, 1.05])

plt.title("All Cohort (Elderly)")
plt.xlabel("Age (Years)")
plt.ylabel("Survival Probability")
plt.legend(loc='best')

ax.text(-0.03, 1.1, "B", transform=ax.transAxes, fontsize=12, fontweight='bold', va='top')

kmf_no = KaplanMeierFitter()
ax = kmf_no.fit(survival_df_60[survival_df_60["ET status"] == 0]["Age"],
                survival_df_60[survival_df_60["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes = KaplanMeierFitter()
ax = kmf_yes.fit(survival_df_60[survival_df_60["ET status"] == 1]["Age"],
                 survival_df_60[survival_df_60["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no, kmf_yes, ax=ax)

# Calculate and annotate median survival times
median_survival_et = kmf_yes.median_survival_time_
median_survival_non_et = kmf_no.median_survival_time_

ax.axhline(y=0.5,xmax=0.5 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=78,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.6 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(survival_df_60[survival_df_60["ET status"] == 0]["Age"],
                       survival_df_60[survival_df_60["ET status"] == 1]["Age"], 
                       event_observed_A=survival_df_60[survival_df_60["ET status"] == 0]["survival_event"], 
                       event_observed_B=survival_df_60[survival_df_60["ET status"] == 1]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}^{digits[1]}")

plt.text(59, 0.3, f"Log-rank\np-value : {logrank_p_value}", fontsize=10)
ax.set_xlabel("Age (Years)")

from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(survival_df_60[["Age","survival_event","ET status"]], duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(59, 0.18, 
         f"HR : {round(hr[0],ndigits=2)}\n$CI_{{95\%}}:$[{round(hr_sum.loc['ET status', 'exp(coef) lower 95%'],ndigits=2)},{round(hr_sum.loc['ET status', 'exp(coef) upper 95%'],ndigits=2)}]", 
         fontsize=10)

# ET+
ax = fig.add_subplot(gs[1, 0])
plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("ET+")

plt.ylabel("Survival Probability")
plt.legend(loc='best')

ax.text(-0.075, 1.1, "C", transform=ax.transAxes, fontsize=12, fontweight='bold', va='top')

kmf_female = KaplanMeierFitter()
ax = kmf_female.fit(et_positive_df[et_positive_df["Sex"] == 1]["Age"],
                et_positive_df[et_positive_df["Sex"] == 1]["survival_event"], 
                label='Female').plot_survival_function(ax=ax,color="#DC0000B2")

kmf_male = KaplanMeierFitter()
ax = kmf_male.fit(et_positive_df[et_positive_df["Sex"] == 2]["Age"],
                 et_positive_df[et_positive_df["Sex"] == 2]["survival_event"], 
                 label='Male').plot_survival_function(ax=ax,color="#4DBBD5B2")

add_at_risk_counts(kmf_female, kmf_male, ax=ax)

# Calculate and annotate median survival times
median_survival_female = kmf_female.median_survival_time_
median_survival_male = kmf_male.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.71 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(et_positive_df[et_positive_df["Sex"] == 1]["Age"],
                       et_positive_df[et_positive_df["Sex"] == 2]["Age"], 
                       event_observed_A=et_positive_df[et_positive_df["Sex"] == 1]["survival_event"], 
                       event_observed_B=et_positive_df[et_positive_df["Sex"] == 2]["survival_event"])


logrank_p_value = round(results.p_value,ndigits=3)

plt.text(39, 0.3, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

from lifelines import CoxPHFitter

cph = CoxPHFitter()
cph.fit(et_positive_cox, duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(39, 0.2, 
         f"HR : {round(hr[0],ndigits=2)}, $CI_{{95\%}}:$[{round(hr_sum.loc['Sex', 'exp(coef) lower 95%'],ndigits=2)},{round(hr_sum.loc['Sex', 'exp(coef) upper 95%'],ndigits=2)}]", 
         fontsize=10)
ax.set_xlabel("Age (Years)")

# ET-
ax = fig.add_subplot(gs[1, 1])
plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("ET-")

plt.ylabel("Survival Probability")
plt.legend(loc='best')

ax.text(-0.075, 1.1, "D", transform=ax.transAxes, fontsize=12, fontweight='bold', va='top')

kmf_female = KaplanMeierFitter()
ax = kmf_female.fit(et_negative_df[et_negative_df["Sex"] == 1]["Age"],
                et_negative_df[et_negative_df["Sex"] == 1]["survival_event"], 
                label='Female').plot_survival_function(ax=ax,color="#DC0000B2")

kmf_male = KaplanMeierFitter()
ax = kmf_male.fit(et_negative_df[et_negative_df["Sex"] == 2]["Age"],
                 et_negative_df[et_negative_df["Sex"] == 2]["survival_event"], 
                 label='Male').plot_survival_function(ax=ax,color="#4DBBD5B2")

add_at_risk_counts(kmf_female, kmf_male, ax=ax)

# Calculate and annotate median survival times
median_survival_female = kmf_female.median_survival_time_
median_survival_male = kmf_male.median_survival_time_

ax.axhline(y=0.5,xmax=0.45 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=79,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.62 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=73,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(et_negative_df[et_negative_df["Sex"] == 1]["Age"],
                       et_negative_df[et_negative_df["Sex"] == 2]["Age"], 
                       event_observed_A=et_negative_df[et_negative_df["Sex"] == 1]["survival_event"], 
                       event_observed_B=et_negative_df[et_negative_df["Sex"] == 2]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}")

plt.text(39, 0.3, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

cph = CoxPHFitter()
cph.fit(et_negative_df[["Age","Sex","survival_event"]], duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(39, 0.2, 
         f"HR : {round(hr[0],ndigits=2)}, $CI_{{95\%}}:$[{round(hr_sum.loc['Sex', 'exp(coef) lower 95%'],ndigits=2)},{round(hr_sum.loc['Sex', 'exp(coef) upper 95%'],ndigits=2)}]", 
         fontsize=10)
ax.set_xlabel("Age (Years)")

# Male
ax = fig.add_subplot(gs[2, 0])
plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("Male")

plt.ylabel("Survival Probability")
plt.legend(loc='best')

ax.text(-0.075, 1.1, "E", transform=ax.transAxes, fontsize=12, fontweight='bold', va='top')

kmf_no_male = KaplanMeierFitter()
ax = kmf_no_male.fit(male_df[male_df["ET status"] == 0]["Age"],
                male_df[male_df["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes_male = KaplanMeierFitter()
ax = kmf_yes_male.fit(male_df[male_df["ET status"] == 1]["Age"],
                 male_df[male_df["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no_male, kmf_yes_male, ax=ax)

# Calculate and annotate median survival times
median_survival_et_male = kmf_yes_male.median_survival_time_
median_survival_non_et_male = kmf_no_male.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=73,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.71 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(male_df[male_df["ET status"] == 1]["Age"],
                       male_df[male_df["ET status"] == 0]["Age"], 
                       event_observed_A=male_df[male_df["ET status"] == 1]["survival_event"], 
                       event_observed_B=male_df[male_df["ET status"] == 0]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}^{digits[1]}")

plt.text(39, 0.38, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

cph = CoxPHFitter(penalizer=0.1)
cph.fit(male_cox, duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(39, 0.23, 
         f"ET(+) HR : {round(hr['Yes'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['Yes', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['Yes', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)

plt.text(39, 0.08, 
         f"ET(-) HR : {round(hr['No'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['No', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['No', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)
ax.set_xlabel("Age (Years)")

# Female
ax = fig.add_subplot(gs[2, 1])
plt.xticks(range(0, max(df["Age"]) + 10, 10))

y_ticks = [0] + [i * 0.25 for i in range(1, 5)]
plt.yticks(y_ticks)

plt.xlim([35, 105])
plt.ylim([0, 1.05])

plt.title("Female")

plt.ylabel("Survival Probability")
plt.legend(loc='best')

ax.text(-0.075, 1.1, "F", transform=ax.transAxes, fontsize=12, fontweight='bold', va='top')

kmf_no_female = KaplanMeierFitter()
ax = kmf_no_female.fit(female_df[female_df["ET status"] == 0]["Age"],
                female_df[female_df["ET status"] == 0]["survival_event"], 
                label='ET-').plot_survival_function(ax=ax,color="#4DBBD5B2")

kmf_yes_female = KaplanMeierFitter()
ax = kmf_yes_female.fit(female_df[female_df["ET status"] == 1]["Age"],
                 female_df[female_df["ET status"] == 1]["survival_event"], 
                 label='ET+').plot_survival_function(ax=ax,color="#DC0000B2")

add_at_risk_counts(kmf_no_female, kmf_yes_female, ax=ax)

# Calculate and annotate median survival times
median_survival_et_female = kmf_yes_female.median_survival_time_
median_survival_non_et_female = kmf_no_female.median_survival_time_

ax.axhline(y=0.5,xmax=0.52 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=79,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')

ax.axhline(y=0.5,xmax=0.71 ,linestyle='-.', color='grey', linewidth=0.8, label='Median')
ax.axvline(x=85,ymax=0.48, linestyle='-.', color='grey', linewidth=0.8, label='Median')


results = logrank_test(female_df[female_df["ET status"] == 1]["Age"],
                       female_df[female_df["ET status"] == 0]["Age"], 
                       event_observed_A=female_df[female_df["ET status"] == 1]["survival_event"], 
                       event_observed_B=female_df[female_df["ET status"] == 0]["survival_event"])

number = results.p_value
exponent = int(np.floor(np.log10(np.abs(number))))
# Calculate the significand (x)
significand = number / 10**exponent
significand = round(significand,ndigits=2)
sign = '-' if exponent < 0 else ''  # Determine the sign
digits = [int(digit) for digit in str(abs(exponent))]  # Get individual digits as integers
logrank_p_value = unicodeit.replace(f"{significand}x10^{sign}^{digits[0]}")

plt.text(39, 0.38, f"Log-rank p-value : {logrank_p_value}", fontsize=10)

cph = CoxPHFitter(penalizer=0.1)
cph.fit(female_cox, duration_col='Age', event_col='survival_event')
hr = cph.hazard_ratios_
hr_sum = cph.summary

plt.text(39, 0.23, 
         f"ET(+) HR : {round(hr['Yes'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['Yes', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['Yes', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)

plt.text(39, 0.08, 
         f"ET(-) HR : {round(hr['No'], ndigits=2)}\n$CI_{{95\%}}$:[{round(hr_sum.loc['No', 'exp(coef) lower 95%'], ndigits=2)},{round(hr_sum.loc['No', 'exp(coef) upper 95%'], ndigits=2)}]",
         fontsize=10)
ax.set_xlabel("Age (Years)")
#fig.suptitle("Survival Analaysis")
plt.savefig('survival_python.png', dpi=300, bbox_inches='tight')
plt.savefig('survival_python.tiff', dpi=300, bbox_inches='tight',pil_kwargs={"compression": "tiff_lzw"})
plt.show()